# msticpy Threat Intel Lookup
This notebook describes the use of the Threat Intelligence lookup class in msticpy.
The class allows lookup of individual or multiple IoCs from one or more TI providers.

TILookup is also extensible - you can subclass TIProvider to implement your own custom lookups. You can also subclass the HTTPProvider or KqlProvider classes, which provide support for querying a REST endpoint or Log Analytics table respectively.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#msticpy-Threat-Intel-Lookup" data-toc-modified-id="msticpy-Threat-Intel-Lookup-1">msticpy Threat Intel Lookup</a></span></li><li><span><a href="#TILookup-class" data-toc-modified-id="TILookup-class-2">TILookup class</a></span><ul class="toc-item"><li><span><a href="#Available-Providers" data-toc-modified-id="Available-Providers-2.1">Available Providers</a></span></li><li><span><a href="#Configuration-File" data-toc-modified-id="Configuration-File-2.2">Configuration File</a></span></li><li><span><a href="#Looking-up-IoCs" data-toc-modified-id="Looking-up-IoCs-2.3">Looking up IoCs</a></span><ul class="toc-item"><li><span><a href="#lookup_ioc" data-toc-modified-id="lookup_ioc-2.3.1">lookup_ioc</a></span></li><li><span><a href="#Lookup-an-IoC-from-a-single-provider" data-toc-modified-id="Lookup-an-IoC-from-a-single-provider-2.3.2">Lookup an IoC from a single provider</a></span><ul class="toc-item"><li><span><a href="#Or-convert-result-to-a-DataFrame-and-let-pandas-do-the-display-work..." data-toc-modified-id="Or-convert-result-to-a-DataFrame-and-let-pandas-do-the-display-work...-2.3.2.1">Or convert result to a DataFrame and let pandas do the display work...</a></span></li></ul></li></ul></li><li><span><a href="#Lookup-using-all-primary-providers" data-toc-modified-id="Lookup-using-all-primary-providers-2.4">Lookup using all primary providers</a></span></li><li><span><a href="#Provider-Usage" data-toc-modified-id="Provider-Usage-2.5">Provider Usage</a></span><ul class="toc-item"><li><span><a href="#Use-to-do-a-passive-DNS-lookup" data-toc-modified-id="Use-to-do-a-passive-DNS-lookup-2.5.1">Use to do a passive DNS lookup</a></span></li></ul></li><li><span><a href="#Inferring-IoC-Type-vs.-Specifying-explicity" data-toc-modified-id="Inferring-IoC-Type-vs.-Specifying-explicity-2.6">Inferring IoC Type vs. Specifying explicity</a></span></li><li><span><a href="#Looking-up-Multiple-IoCs" data-toc-modified-id="Looking-up-Multiple-IoCs-2.7">Looking up Multiple IoCs</a></span><ul class="toc-item"><li><span><a href="#lookup_iocs" data-toc-modified-id="lookup_iocs-2.7.1">lookup_iocs</a></span></li><li><span><a href="#Multiple-IoCs-using-all-providers" data-toc-modified-id="Multiple-IoCs-using-all-providers-2.7.2">Multiple IoCs using all providers</a></span></li></ul></li><li><span><a href="#Specifying-Time-Ranges" data-toc-modified-id="Specifying-Time-Ranges-2.8">Specifying Time Ranges</a></span></li></ul></li></ul></div>

In [1]:
# Imports
import sys
import warnings

from msticpy.nbtools.utility import check_py_version
MIN_REQ_PYTHON = (3,6)
check_py_version(MIN_REQ_PYTHON)

from IPython import get_ipython
from IPython.display import display, HTML, Markdown

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

from msticpy.data import QueryProvider
from msticpy.nbtools import *
from msticpy.sectools import *


%xmode Verbose

Exception reporting mode: Verbose


# TILookup class
Input can be a single IoC observable or a pandas DataFrame containing
multiple observables. Processing may require a an API key and
processing performance may be limited to a specific number of
requests per minute for the account type that you have.

In [2]:
# TILookup class
display(Markdown("### Constructor\n"))
print(TILookup.__init__.__doc__)
display(Markdown("### Attributes\n"))
for name in [att for att in dir(TILookup) if not att.startswith("_")]:
    display(Markdown(f"#### _{name}()_"))
    print(getattr(TILookup, name).__doc__)
    print()

### Constructor



        Initialize TILookup instance.

        Parameters
        ----------
        primary_providers : Optional[List[TIProvider]], optional
            Primary TI Providers, by default None
        secondary_providers : Optional[List[TIProvider]], optional
            Secondary TI Providers, by default None

        


### Attributes


#### _add_provider()_


        Add a TI provider to the current collection.

        Parameters
        ----------
        provider : TIProvider
            Provider instance
        name : str, optional
            The name to use for the provider (overrides the class name
            of `provider`)
        primary : bool, optional
            "primary" or "secondary" if False, by default "primary"

        



...

Reload provider settings from config.



#### _reload_providers()_

Reload providers based on currret settings in config.



#### _result_to_df()_


        Return DataFrame representation of IoC Lookup response.

        Parameters
        ----------
        ioc_lookup : Tuple[bool, List[Tuple[str, LookupResult]]]
            Output from `lookup_ioc`

        Returns
        -------
        pd.DataFrame
            The response as a DataFrame with a row for each
            provider response.

        



## Available Providers
The **msticpy** TI Provider library can lookup IoCs in multiple providers.

The list below shows the current set of providers.

In [3]:
ti_lookup = TILookup()
# List available providers
ti_lookup.available_providers

['AzSTI', 'OTX', 'VirusTotal', 'XForce']

## Configuration File
You can configure primary and secondary providers. 
Primary providers are used by default.

You may need to supply an authorization (API) key and in some cases a user ID for each provider.

For LogAnalytics/Azure Sentinel providers, you will need the workspace ID and tenant ID and will need to authenticate in order to access the data (although if you have an existing authenticated connection with the same workspace/tenant, this connection will be re-used).

The configuration file is read from the current directory.

Alternatively, you can specify a location for this file in an environment variable `MSTICPYCONFIG`.

If you need to create a config file, uncomment the lines in the following cell.<br>
<font color="red">Warning - this will overwrite a file of the same name in the current directory</font>

Delete any provider entries that you do not want to use and add the missing parameters for your providers. 

In [5]:
# %%writefile msticpyconfig.yaml
# QueryDefinitions:

# TIProviders:
#   OTX:
#     Args:
#       AuthKey: "your-otx-key"
#     Primary: True
#     Provider: "OTX" # Explicitly name provider to override
#   VirusTotal:
#     Args:
#       AuthKey: "your-vt-key"
#     Primary: True
#     Provider: "VirusTotal"
#   XForce:
#     Args:
#       ApiID: "your-xforce-id"
#       AuthKey: "your-xforce-key"
#     Primary: True
#     Provider: "XForce"
#   AzureSentinel:
#     Args:
#       WorkspaceID: "your-azure-sentinel-workspace-id"
#       TenantID: "your-azure-sentinel-tenant-id"
#     Primary: True
#     Provider: "AzSTI"

In [6]:
ti_lookup.reload_providers()
ti_lookup.provider_status

<IPython.core.display.Javascript object>

['OTX - AlientVault OTX Lookup. (primary)',
 'VirusTotal - VirusTotal Lookup. (primary)',
 'XForce - IBM XForce Lookup. (primary)',
 'AzSTI - Azure Sentinel TI provider class. (primary)']

## Looking up IoCs

### lookup_ioc
To lookup a single IoC.
```
ti_lookup.lookup_ioc(
    observable: str = None,
    ioc_type: str = None,
    ioc_query_type: str = None,
    providers: List[str] = None,
    prov_scope: str = 'primary',
    **kwargs,
) -> Tuple[bool, List[Tuple[str, msticpy.sectools.tiproviders.ti_provider_base.LookupResult]]]

Lookup single IoC in active providers.

Parameters
----------
observable : str
    IoC observable
    (`ioc` is also an alias for observable)
ioc_type : str, optional
    One of IoCExtract.IoCType, by default None
    If none, the IoC type will be inferred
ioc_query_type: str, optional
    The ioc query type (e.g. rep, info, malware)
providers: List[str]
    Explicit list of providers to use
prov_scope : str, optional
    Use primary, secondary or all providers, by default "primary"
kwargs :
    Additional arguments passed to the underlying provider(s)

Returns
-------
Tuple[bool, List[Tuple[str, LookupResult]]]
    The result returned as a tuple(bool, list):
    bool indicates whether a TI record was found in any provider
    list has an entry for each provider result
```

In [7]:
ti_lookup.lookup_ioc?

### Lookup an IoC from a single provider
And show the output

In [8]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

result, details = ti_lookup.lookup_ioc(observable="38.75.137.9", providers=["OTX"])

print("Positive" if result else "Negative")
# the details is a list (since there could be multiple responses for an IoC)
for provider, detail in details:
    print(provider)
    detail.summary
    print("\nRaw Results")
    pp.pprint(detail.raw_result)


Positive
OTX
ioc: 38.75.137.9 ( ipv4 )
result: True
{   'names': ['Underminer EK'],
    'pulse_count': 1,
    'references': [   [   'https://blog.malwarebytes.com/threat-analysis/2019/07/exploit-kits-summer-2019-review/']],
    'tags': [[]]}
reference:  https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/general

Raw Results
{ 'area_code': 0,
  'asn': 'AS63023 GTHost',
  'base_indicator': { 'access_reason': '',
                      'access_type': 'public',
                      'content': '',
                      'description': '',
                      'id': 2127020821,
                      'indicator': '38.75.137.9',
                      'title': '',
                      'type': 'IPv4'},
  'charset': 0,
  'city': 'Los Angeles',
  'city_data': True,
  'continent_code': 'NA',
  'country_code': 'US',
  'country_code3': 'USA',
  'country_name': 'United States',
  'dma_code': 803,
  'flag_title': 'United States',
  'flag_url': '/assets/images/flags/us.png',
  'indicator': '

#### Or convert result to a DataFrame and let pandas do the display work...

In [9]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9", providers=["OTX"])
ti_lookup.result_to_df(result).T

,OTX
IoC,38.75.137.9
IoCType,ipv4
QuerySubtype,None
Result,True
Details,"{'pulse_count': 1, 'names': ['Underminer EK'], 'tags': [[]], 'references': [['https://blog.malwa..."
RawResult,"{'sections': ['general', 'geo', 'reputation', 'url_list', 'passive_dns', 'malware', 'nids_list',..."
Reference,https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/general
Status,200


In [10]:
# Extract a single field (RawResult) from the dataframe (.iloc[0] is to select the row)
ti_lookup.result_to_df(result)["RawResult"].iloc[0]

{'sections': ['general',
  'geo',
  'reputation',
  'url_list',
  'passive_dns',
  'malware',
  'nids_list',
  'http_scans'],
 'city': 'Los Angeles',
 'area_code': 0,
 'pulse_info': {'count': 1,
  'references': ['https://blog.malwarebytes.com/threat-analysis/2019/07/exploit-kits-summer-2019-review/'],
  'pulses': [{'indicator_type_counts': {'URL': 16,
     'FileHash-MD5': 5,
     'IPv4': 3},
    'pulse_source': 'web',
    'TLP': 'white',
    'description': '',
    'subscriber_count': 10,
    'tags': [],
    'export_count': 0,
    'malware_families': [],
    'is_modified': False,
    'upvotes_count': 0,
    'modified_text': '19 days ago ',
    'is_subscribing': None,
    'references': ['https://blog.malwarebytes.com/threat-analysis/2019/07/exploit-kits-summer-2019-review/'],
    'targeted_countries': [],
    'groups': [{'name': 'DCT Security Team', 'id': 614}],
    'vote': 0,
    'validator_count': 0,
    'threat_hunter_scannable': True,
    'is_author': False,
    'adversary': '',
    

## Lookup using all primary providers

In [11]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9")
ti_lookup.result_to_df(result)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Warning - query did not complete successfully.
Kql ResultSet returned - check  'completion_query_info' property.


,IoC,IoCType,QuerySubtype,Result,Details,RawResult,Reference,Status
OTX,38.75.137.9,ipv4,None,True,"{'pulse_count': 1, 'names': ['Underminer EK'], 'tags': [[]], 'references': [['https://blog.malwa...","{'sections': ['general', 'geo', 'reputation', 'url_list', 'passive_dns', 'malware', 'nids_list',...",https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/general,200
VirusTotal,38.75.137.9,ipv4,None,True,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'detected_urls': ['http://38.75.137...","{'asn': 63023, 'undetected_urls': [['http://38.75.137.9:9088/', '3d5edb0e0bb726e414a9b76dac619c1...",https://www.virustotal.com/vtapi/v2/ip-address/report,200
XForce,38.75.137.9,ipv4,None,True,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '38.75.137.9', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional In...",https://api.xforce.ibmcloud.com/ipr/38.75.137.9,200
AzSTI,38.75.137.9,ipv4,None,False,0 rows returned.,None,None,-1


## Provider Usage
This shows the supported IoC Types.

In some cases an IoC type will also support special types of sub-query such as geo-ip and passive-dns

In [22]:
ti_lookup.provider_usage()

Primary providers
-----------------

Provider class: OTX
AlientVault OTX Lookup. Supported query types:
	ioc_type=dns
	ioc_type=dns, ioc_query_type=geo
	ioc_type=dns, ioc_query_type=passivedns
	ioc_type=file_hash
	ioc_type=hostname
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=geo
	ioc_type=ipv4, ioc_query_type=passivedns
	ioc_type=ipv6
	ioc_type=ipv6, ioc_query_type=geo
	ioc_type=ipv6, ioc_query_type=passivedns
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url

Provider class: VirusTotal
VirusTotal Lookup. Supported query types:
	ioc_type=dns
	ioc_type=file_hash
	ioc_type=ipv4
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url

Provider class: XForce
IBM XForce Lookup. Supported query types:
	ioc_type=dns, ioc_query_type=info
	ioc_type=dns, ioc_query_type=passivedns
	ioc_type=dns, ioc_query_type=whois
	ioc_type=file_hash
	ioc_type=hostname, ioc_query_type=whois
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=malware
	ioc_type=ipv4, ioc_qu

### Use to do a passive DNS lookup

In [13]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9", ico_type="ipv4", ioc_query_type="passivedns", providers=["XForce"])
result

(True,
 [('XForce',
   LookupResult(ioc='38.75.137.9', ioc_type='ipv4', query_subtype='passivedns', result=True, details={'records': 1}, raw_result={'Passive': {'query': '0x00000000000000000000ffff264b8909', 'records': []}, 'RDNS': ['9-137-75-38.clients.gthost.com'], 'total_rows': 1}, reference='https://api.xforce.ibmcloud.com/resolve/38.75.137.9', status=200))])

## Inferring IoC Type vs. Specifying explicity
If you do a lookup without specifying a type, TILookup will try to infer the type by matching regexes. There are patterns for all supported types but there are some caveats:

- The match is not 100% foolproof - e.g. some URLs and hash types may be misidentified.
- The inference adds an overhead to each lookup.

If you know the type that you want to look up, it is always better to explicitly include it.
- For single IoC lookup, use the `ioc_type` parameter.
- For multiple IoC lookups (see below), supply either:
  - a DataFrame with a column that specifies the type for each entry
  - a dictionary of the form `{ioc_observable: ioc_type}`

## Looking up Multiple IoCs

### lookup_iocs
```
Signature:
ti_lookup.lookup_iocs(
    data: Union[pandas.core.frame.DataFrame, Mapping[str, str], Iterable[str]],
    obs_col: str = None,
    ioc_type_col: str = None,
    ioc_query_type: str = None,
    providers: List[str] = None,
    prov_scope: str = 'primary',
    **kwargs,
) -> pandas.core.frame.DataFrame

Lookup a collection of IoCs.

Parameters
----------
data : Union[pd.DataFrame, Mapping[str, str], Iterable[str]]
    Data input in one of three formats:
    1. Pandas dataframe (you must supply the column name in
    `obs_col` parameter)
    2. Mapping (e.g. a dict) of [observable, IoCType]
    3. Iterable of observables - IoCTypes will be inferred
obs_col : str, optional
    DataFrame column to use for observables, by default None
ioc_type_col : str, optional
    DataFrame column to use for IoCTypes, by default None
ioc_query_type: str, optional
    The ioc query type (e.g. rep, info, malware)
providers: List[str]
    Explicit list of providers to use
prov_scope : str, optional
    Use primary, secondary or all providers, by default "primary"
kwargs :
    Additional arguments passed to the underlying provider(s)

Returns
-------
pd.DataFrame
    DataFrame of results
```

In [14]:
# View the docstring (as above)
ti_lookup.lookup_iocs?

In [15]:
ioc_ips = [
    "185.92.220.35",
    "213.159.214.86",
    "77.222.54.202",
    "91.219.29.81",
    "193.9.28.254",
    "89.108.83.196",
    "91.219.28.44",
    "188.127.231.124",
    "192.42.116.41",
    "91.219.31.18",
    "46.4.239.76",
    "188.166.168.250",
    "195.154.241.208",
    "51.255.172.55",
    "93.170.169.52",
    "104.215.148.63",
    "13.77.161.179",
    "40.76.4.15",
    "40.112.72.205",
    "40.113.200.201",
]

ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,IoC,IoCType,QuerySubtype,Reference,Result,Status,Details,RawResult,Provider
0,213.159.214.86,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '0164ADB4A6CB7A79FBAE7BE90A43050B090A18364E3855048AC86B9DA5E0A92B', 'TimeGenerat...",AzSTI
1,40.113.200.201,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,False,-1.0,0 rows returned.,NaN,AzSTI
2,91.219.29.81,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '3F458D91A21866C9037B99D997379A6906573766C0C2F8FB45327A6A15676A0D', 'TimeGenerat...",AzSTI
3,89.108.83.196,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'C3CA82D5B30A34F4BD6188C9DCFAD9E46D3C0CC45CC4FD969DA3A398DC34B1AE', 'TimeGenerat...",AzSTI
4,192.42.116.41,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '2F321C9D2593B6EF59DEB64B6CB209F375529C429F0DF463D639784E7353AA5D', 'TimeGenerat...",AzSTI
5,13.77.161.179,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,False,-1.0,0 rows returned.,NaN,AzSTI
6,91.219.28.44,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '34873942B59A0E9D9526F6BAEE8C1CDC4BA4D25B24183758DCFA75BC84E82154', 'TimeGenerat...",AzSTI
7,91.219.31.18,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'E544978153B13A50440A8FA7A9107BE8B20BBAEB3827BCF7CFEBAD19E36A725C', 'TimeGenerat...",AzSTI
8,40.76.4.15,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,False,-1.0,0 rows returned.,NaN,AzSTI
9,93.170.169.52,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:32.934234Z) | w...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'B4A04178B747900F47F902D057010F39DB585EAFEA3729EC87359AE277D90392', 'TimeGenerat...",AzSTI


### Multiple IoCs using all providers
Output sorted by IoC

In [16]:
ioc_urls = [
    "http://cheapshirts.us/zVnMrG.php",
    "http://chinasymbolic.com/i9jnrc",
    "http://cetidawabi.com/468fd",
    "http://append.pl/srh9xsz",
    "http://aiccard.co.th/dvja1te",
    "http://ajaraheritage.ge/g7cberv",
    "http://cic-integration.com/hjy93JNBasdas",
    "https://google.com",  # benign
    "https://microsoft.com",  # benign
    "https://python.org",  # benign
]
results = ti_lookup.lookup_iocs(data=ioc_urls)
results.sort_values("IoC")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,IoC,IoCType,QuerySubtype,Result,Details,RawResult,Reference,Provider,Status
0,http://aiccard.co.th/dvja1te,url,None,True,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'FAE39C007D6554822504A1E0BDFD788E27DDC748ED63B258651DE52E4FA6D511', 'TimeGenerat...",ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:41.900764Z) | w...,AzSTI,0.0
4,http://aiccard.co.th/dvja1te,url,None,True,"{'cats': None, 'categoryDescriptions': None}","{'result': {'url': 'aiccard.co.th', 'cats': {}, 'score': None, 'categoryDescriptions': {}}, 'ass...",https://api.xforce.ibmcloud.com/url/http://aiccard.co.th/dvja1te,XForce,NaN
4,http://aiccard.co.th/dvja1te,url,None,True,"{'pulse_count': 3, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)', ...","{'indicator': 'http://aiccard.co.th/dvja1te', 'alexa': 'http://www.alexa.com/siteinfo/aiccard.co...",https://otx.alienvault.com/api/v1/indicators/url/http://aiccard.co.th/dvja1te/general,OTX,NaN
4,http://aiccard.co.th/dvja1te,url,None,False,No response from provider.,<Response [403]>,https://www.virustotal.com/vtapi/v2/url/report,VirusTotal,NaN
5,http://ajaraheritage.ge/g7cberv,url,None,True,"{'cats': None, 'categoryDescriptions': None}","{'result': {'url': 'ajaraheritage.ge', 'cats': {}, 'score': None, 'categoryDescriptions': {}}, '...",https://api.xforce.ibmcloud.com/url/http://ajaraheritage.ge/g7cberv,XForce,NaN
5,http://ajaraheritage.ge/g7cberv,url,None,True,"{'pulse_count': 2, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)', ...","{'indicator': 'http://ajaraheritage.ge/g7cberv', 'alexa': 'http://www.alexa.com/siteinfo/ajarahe...",https://otx.alienvault.com/api/v1/indicators/url/http://ajaraheritage.ge/g7cberv/general,OTX,NaN
7,http://ajaraheritage.ge/g7cberv,url,None,True,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '9DA8A022147F28483F2F9C4DF138AC54458764F3C83260377229803C82085C12', 'TimeGenerat...",ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:41.900764Z) | w...,AzSTI,0.0
5,http://ajaraheritage.ge/g7cberv,url,None,False,No response from provider.,<Response [403]>,https://www.virustotal.com/vtapi/v2/url/report,VirusTotal,NaN
3,http://append.pl/srh9xsz,url,None,True,"{'cats': None, 'categoryDescriptions': None}","{'result': {'url': 'append.pl', 'cats': {'Software / Hardware': True}, 'score': 1, 'categoryDesc...",https://api.xforce.ibmcloud.com/url/http://append.pl/srh9xsz,XForce,NaN
1,http://append.pl/srh9xsz,url,None,True,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '7EE248A07BF627B3C706834ECAB1FFF2235E0E8D4811B9DD19AB2A33984D90E3', 'TimeGenerat...",ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-21T17:30:41.900764Z) | w...,AzSTI,0.0


## Specifying Time Ranges
Some providers (currently only AzSTI) support time ranges so that you can specify specific periods to search for.

If a provider does not support time ranges, the parameters will be ignored

In [17]:
from datetime import datetime
search_origin = datetime(2019, 8, 5)
q_times = nbwidgets.QueryTime(units="hour", auto_display=True, origin_time=search_origin, max_after=24, max_before=24)

HTML(value='<h4>Set query time boundaries</h4>')

In [19]:
# Using this data range returned no results
ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI", start=q_times.start, end=q_times.end).head()

<IPython.core.display.Javascript object>

Warning - query did not complete successfully.
Kql ResultSet returned - check  'completion_query_info' property.


,IoC,IoCType,QuerySubtype,Reference,Result,Details,Status,Provider
0,213.159.214.86,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-08-04T00:00:00Z) | where Ti...,False,0 rows returned.,-1,AzSTI
1,40.113.200.201,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-08-04T00:00:00Z) | where Ti...,False,0 rows returned.,-1,AzSTI
2,91.219.29.81,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-08-04T00:00:00Z) | where Ti...,False,0 rows returned.,-1,AzSTI
3,89.108.83.196,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-08-04T00:00:00Z) | where Ti...,False,0 rows returned.,-1,AzSTI
4,192.42.116.41,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-08-04T00:00:00Z) | where Ti...,False,0 rows returned.,-1,AzSTI


In [20]:
from datetime import datetime
search_origin = datetime(2019, 8, 5)
q_times = nbwidgets.QueryTime(units="day", auto_display=True, origin_time=search_origin, max_after=24, max_before=24)

HTML(value='<h4>Set query time boundaries</h4>')

In [21]:
# Using a wider ranges produces results
ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI", start=q_times.start, end=q_times.end)

,IoC,IoCType,QuerySubtype,Reference,Result,Status,Details,RawResult,Provider
0,213.159.214.86,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '0164ADB4A6CB7A79FBAE7BE90A43050B090A18364E3855048AC86B9DA5E0A92B', 'TimeGenerat...",AzSTI
1,40.113.200.201,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,False,-1.0,0 rows returned.,NaN,AzSTI
2,91.219.29.81,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '3F458D91A21866C9037B99D997379A6906573766C0C2F8FB45327A6A15676A0D', 'TimeGenerat...",AzSTI
3,89.108.83.196,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'C3CA82D5B30A34F4BD6188C9DCFAD9E46D3C0CC45CC4FD969DA3A398DC34B1AE', 'TimeGenerat...",AzSTI
4,192.42.116.41,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '2F321C9D2593B6EF59DEB64B6CB209F375529C429F0DF463D639784E7353AA5D', 'TimeGenerat...",AzSTI
5,13.77.161.179,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,False,-1.0,0 rows returned.,NaN,AzSTI
6,91.219.28.44,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': '34873942B59A0E9D9526F6BAEE8C1CDC4BA4D25B24183758DCFA75BC84E82154', 'TimeGenerat...",AzSTI
7,91.219.31.18,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'E544978153B13A50440A8FA7A9107BE8B20BBAEB3827BCF7CFEBAD19E36A725C', 'TimeGenerat...",AzSTI
8,40.76.4.15,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,False,-1.0,0 rows returned.,NaN,AzSTI
9,93.170.169.52,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2019-07-12T00:00:00Z) | where Ti...,True,0.0,"{'Action': 'alert', 'ThreatType': 'Malware', 'ThreatSeverity': nan, 'Active': True, 'Description...","{'IndicatorId': 'B4A04178B747900F47F902D057010F39DB585EAFEA3729EC87359AE277D90392', 'TimeGenerat...",AzSTI


In [4]:
q_times.start

datetime.datetime(2019, 8, 4, 0, 0)

In [9]:
qry_prov.ThreatIntelligence.list_indicators("print", observables="foo")

' ThreatIntelligenceIndicator  | where TimeGenerated >= datetime(2019-08-14T22:22:00.174979Z) | where TimeGenerated <= datetime(2019-08-15T22:22:00.174979Z) | summarize arg_max(TimeGenerated, *) by IndicatorId '

In [8]:
qry_prov.ThreatIntelligence.list_indicators_by_url(observables="http://37.49.225.241/bins/gemini.mpsl")

,IndicatorId,TimeGenerated,TenantId,SourceSystem,Action,ActivityGroupNames,AdditionalInformation,ApplicationId,AzureTenantId,ConfidenceScore,Description,DiamondModel,ExternalIndicatorId,ExpirationDateTime,ThreatType,Active,KillChainActions,KillChainC2,KillChainDelivery,KillChainExploitation,KillChainReconnaissance,KillChainWeaponization,KnownFalsePositives,MalwareNames,PassiveOnly,...,FileHashValue,FileMutexName,FileName,FilePacker,FilePath,FileSize,FileType,DomainName,NetworkIP,NetworkPort,NetworkDestinationAsn,NetworkDestinationCidrBlock,NetworkDestinationIP,NetworkCidrBlock,NetworkDestinationPort,NetworkProtocol,NetworkSourceAsn,NetworkSourceCidrBlock,NetworkSourceIP,NetworkSourcePort,Url,UserAgent,IndicatorProvider,Type,IoC
0,8BA46D32ECA00EAA545C460C0BF381CD8DF534BDC03F97EC5D4EFB0ABF06F659,2019-08-15 18:42:04.142,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from urlhaus.URL,,URL:http://37.49.225.241/bins/gemini.mpsl,2019-09-13 18:42:02.892,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://37.49.225.241/bins/gemini.mpsl,,,ThreatIntelligenceIndicator,http://37.49.225.241/bins/gemini.mpsl


In [15]:
_kql_raw_result_.query

'ThreatIntelligenceIndicator  | where TimeGenerated >= datetime(2019-08-14T22:22:29.859783Z) | where TimeGenerated <= datetime(2019-08-15T22:22:29.859783Z) | summarize arg_max(TimeGenerated, *) by IndicatorId'

In [2]:
%pdb on
%xmode Verbose

Automatic pdb calling has been turned ON
Exception reporting mode: Verbose


In [5]:

ti_lookup = TILookup()

<IPython.core.display.Javascript object>

In [8]:
as_prov = ti_lookup.loaded_providers["AzSTI"]
as_prov._query_provider.list_data_environments()
# all_queries',
#  'connect',
#  'connected',
#  'connection_string',
#  'exec_query',
#  'import_query_file',
#  'list_data_environments',
#  'list_queries',
#  'query_help']

['LogAnalytics', 'Kusto', 'AzureSecurityCenter', 'SecurityGraph']

In [9]:
# %%debug
ti_lookup.lookup_ioc(observable="http://37.49.225.241/bins/gemini.mpsl", providers=["AzSTI"])

(True,
 [('AzSTI',
   LookupResult(ioc='http://37.49.225.241/bins/gemini.mpsl', ioc_type='url', query_subtype=None, result=True, details={'Action': ['alert'], 'ThreatType': ['Malware'], 'ThreatSeverity': [], 'Active': [True], 'Description': ['URL indicator from urlhaus.URL'], 'ConfidenceScore': [100.0]}, raw_result=                                                        IndicatorId  \
   0  8BA46D32ECA00EAA545C460C0BF381CD8DF534BDC03F97EC5D4EFB0ABF06F659   
   
               TimeGenerated                              TenantId SourceSystem  \
   0 2019-08-15 18:42:04.142  a927809c-8142-43e1-96b3-4ad87cfe95a3    Indicator   
   
     Action ActivityGroupNames AdditionalInformation  \
   0  alert                 []                         
   
                             ApplicationId                         AzureTenantId  \
   0  96B550D5-48A2-41FB-B42E-2E9F9C57E73C  69d28fd7-42a5-48bc-a619-af56397b9f28   
   
      ConfidenceScore                     Description DiamondModel  \
   0  

In [5]:
combo_df = pd.read_pickle("as_threatintel")

In [19]:
# combo_df[combo_df["Url"] == "http://ajaraheritage.ge/g7cberv"]
combo_df["NetworkIP"].values

array(['51.255.107.37', '185.22.67.27', '37.235.53.210', '51.255.107.8',
       '95.213.195.123', '91.200.14.109', '5.187.5.171', '193.0.178.74',
       '45.56.77.175', '185.127.25.176', '185.75.46.122', '193.9.28.49',
       '37.235.50.29', '31.184.233.106', '185.162.8.101', '46.8.45.18',
       '91.219.31.15', '91.230.211.103', '185.5.250.135', '51.254.240.89',
       '185.117.153.176', '92.63.87.53', '92.63.87.134', '46.148.20.46',
       '217.106.238.89', '31.41.44.130', '89.108.85.163', '83.217.8.155',
       '93.170.123.60', '88.198.119.177', '95.46.114.205', '95.46.114.97',
       '83.217.11.191', '194.1.236.126', '207.244.97.230',
       '91.230.211.26', '176.31.127.168', '217.12.218.158', '31.41.44.45',
       '195.43.95.198', '216.218.135.114', '5.34.183.195', '5.135.76.18',
       '91.239.235.130', '188.138.88.184', '92.63.87.48', '91.226.92.208',
       '194.58.56.189', '31.41.47.41', '185.102.136.67', '91.219.29.41',
       '95.85.19.195', '195.22.28.196', '194.58.56.200',

In [19]:
ind_df = qry_prov.ThreatIntelligence.list_indicators(start=q_times.start, end=q_times.end)


,IndicatorId,TimeGenerated,TenantId,SourceSystem,Action,ActivityGroupNames,AdditionalInformation,ApplicationId,AzureTenantId,ConfidenceScore,Description,DiamondModel,ExternalIndicatorId,ExpirationDateTime,ThreatType,Active,KillChainActions,KillChainC2,KillChainDelivery,KillChainExploitation,KillChainReconnaissance,KillChainWeaponization,KnownFalsePositives,MalwareNames,PassiveOnly,...,FileHashType,FileHashValue,FileMutexName,FileName,FilePacker,FilePath,FileSize,FileType,DomainName,NetworkIP,NetworkPort,NetworkDestinationAsn,NetworkDestinationCidrBlock,NetworkDestinationIP,NetworkCidrBlock,NetworkDestinationPort,NetworkProtocol,NetworkSourceAsn,NetworkSourceCidrBlock,NetworkSourceIP,NetworkSourcePort,Url,UserAgent,IndicatorProvider,Type
0,11A9F4CBCCC81E36D0C2A41452CE35EE457955D62C0AF241BABD55A7C2CBC420,2019-08-05 22:49:49.926,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://colortown.pt/ldjivy78,2019-09-03 22:16:28.552,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://colortown.pt/ldjivy78,,,ThreatIntelligenceIndicator
1,0D9377F9C17DB9CDD9BE14DEDDF5E8FE0B4B035158E589B545CE1F9B8A5CC92C,2019-08-05 22:18:24.413,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,70.0,IPv4 indicator from ransomwaretracker.RW_IPBL,,IPv4:51.255.107.37,2019-09-03 22:16:15.923,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,51.255.107.37,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,,,,ThreatIntelligenceIndicator
2,E3CD437E0B19958893D9A0560AA05C56A041D9FD9A7C30F97F0AD8B90809FB53,2019-08-05 22:55:27.992,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://download.perle.com/09y7gnninj,2019-09-03 22:16:30.556,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://download.perle.com/09y7gnninj,,,ThreatIntelligenceIndicator
3,1DF7139C9B662E9CDB3372D313B2DA74D90486C4185C704D3B57D40EA8E40853,2019-08-05 22:56:59.970,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://ecofloraholland.nl/JGHldb03m,2019-09-03 22:16:30.840,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://ecofloraholland.nl/JGHldb03m,,,ThreatIntelligenceIndicator
4,07ABD3C13D8F299D3CA47EBD0241D089787AC11CA211322D59E6C282DA1D4588,2019-08-05 22:51:04.201,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://d2dlaundry.com/i9jnrc,2019-09-03 22:16:28.849,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://d2dlaundry.com/i9jnrc,,,ThreatIntelligenceIndicator
5,72D571BADF6B3F9BD45489A915EF0B8DE854CC167FCA94F385DF52D2D7E5CF24,2019-08-05 22:59:59.809,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://esbook.com/phsys.php,2019-09-03 22:16:32.326,Malware,True,None,None,None,None,None,None,,[],None,...,,,,,,,NaN,,,,NaN,NaN,,,,NaN,NaN,NaN,,,NaN,http://esbook.com/phsys.php,,,ThreatIntelligenceIndicator
6,5FB8A0F196FD44906EEAE9C64DD1E7E4734D751B5859011058250F5414D3D056,2019-08-05 22:32:47.116,a927809c-8142-43e1-96b3-4ad87cfe95a3,Indicator,alert,[],,96B550D5-48A2-41FB-B42E-2E9F9C57E73C,69d28fd7-42a5-48bc-a619-af56397b9f28,100.0,URL indicator from ransomwaretracker.RW_URLBL,,URL:http://arttoday.sk/mE8MKJ.php,2019-09-03 2

In [31]:
ip_df = ind_df[ind_df['NetworkIP'].str.len() > 0]
url_df = ind_df[ind_df['Url'].str.len() > 0]
comb_df = pd.concat([ip_df, url_df[0:300]])
comb_df.to_pickle("as_threatintel")

In [13]:
end = combo_df["TimeGenerated"].max()
start = combo_df["TimeGenerated"].min()
start.to_pydatetime()

datetime.datetime(2019, 8, 5, 22, 16, 16, 574000)

In [22]:
from itertools import chain

def _series_to_list(series: pd.Series):
   return list({val for val in series.dropna().values})

_series_to_list(ind_df["IndicatorId"])

['...']

In [10]:

def is_abstract(cls):
    return bool(getattr(cls, "__abstractmethods__", False))

# from tiproviders import TIProvider
import abc
dir(tiproviders)
for provider_name in dir(tiproviders):
    provider_class = getattr(
                    tiproviders, provider_name, None
                )
    if not provider_class or not isclass(provider_class):
        continue
    if issubclass(provider_class, tiproviders.TIProvider) and not is_abstract(provider_class):
        print("Found class", provider_class.__name__)

Found class AzSTI
Found class OTX
Found class VirusTotal
Found class XForce


In [19]:
list({val for val in ind_df["ThreatSeverity"].dropna().values})

[]